In [1]:
## Sophia Hart
## Title: Green Technology Agents
### Subtitle: Research Agents does the latest research and provides a summary. It has memory capability and saves tokens.
## Google 5-Day Agentic AI Intensive Capstone Project
## Nov 2025

### Session 1: Setup

#### 1.1 Authenticate in the Notebook
Before running the cell below, enter GOOGLE_API_KEY in the Secrets under Add-ons.
#### IMPORTANT: IF RUNNING DEPLOYMENT, DON'T RUN THIS CELL. GO STRAIGHT TO SESSION 4 ON DEPLOYMENT.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


#### 1.2 Import ADK components

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

import os
import random
import time
import vertexai
from kaggle_secrets import UserSecretsClient
from vertexai import agent_engines

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


#### 1.3 Configure retry option

In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

### Session 2: Architecture

### 2.1 Multi-agent

**1. research_agent**: Find most updated research findings using Google Search.

**2. summary_agent**: Summary findings from Rearch Agent

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [5]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


#### 2.2 Orchestration 

In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


### Session 3: Agent Memory


#### 3.1: Helper functions
This helper function manages a complete conversation session, handling session creation/retrieval, query processing, and response streaming.

In [7]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


#### 3.2 Three-step memory integration process:

**Initialize** → Create a MemoryService and provide it to your agent via the Runner

**Ingest** → Transfer session data to memory using add_session_to_memory()

**Retrieve** → Search stored memories using search_memory()

#### a. Initialize Memory

In [8]:
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

In [9]:
# Define constants used throughout the notebook
APP_NAME = "ResearchMemoryApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResearchMemoryAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[AgentTool(agent=root_agent), 
           load_memory],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("✅ Agent with load_memory tool created.")

✅ Agent with load_memory tool created.


#### b. Create Runner¶
Now provide both Session and Memory services to the Runner.

Key configuration:

The Runner requires both services to enable memory functionality:

**session_service** → Manages conversation threads and events

**memory_service** → Provides long-term knowledge storage

Both services work together: Sessions capture conversations, Memory stores knowledge for retrieval across sessions.

In [10]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="ResearchMemoryApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


#### c. Create a Session

In [11]:
# User tells agent about their favorite color
await run_session(
    runner,
    "I'm interested in green technology, especially fusion, fission, and battery technologies. What are the latest advancements in sustainable energy?",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > I'm interested in green technology, especially fusion, fission, and battery technologies. What are the latest advancements in sustainable energy?


Model: > Recent breakthroughs in sustainable energy include advancements in fusion energy research, particularly in understanding plasma behavior through sophisticated modeling and international collaborations. In fission energy, China is leading the expansion with new reactor designs, and Small Modular Reactors (SMRs) are being developed for better cost-efficiency and safety. For battery technologies, there's rapid progress in areas like sodium-ion, solid-state, and graphene batteries, aiming to improve efficiency, reduce costs, and enhance safety for applications ranging from grid storage to electric vehicles.


Let's verify the conversation is captured in the session.

In [12]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("📝 Session contains:")
for event in session.events:
    # 1. Start with the default empty text
    text = "(empty)"
    
    # 2. Check if content exists
    if event.content:
        # 3. Check if 'parts' list exists AND is not empty
        if event.content.parts and len(event.content.parts) > 0:
            part = event.content.parts[0]
            
            # 4. Check if the 'text' attribute exists on the part
            if hasattr(part, 'text') and part.text:
                text = part.text[:60]
            # Handle the case where the part is a non-text type (e.g., function call, image, etc.)
            else:
                 text = f"({part.type if hasattr(part, 'type') else 'Non-Text Part'})..."

    print(f"  {event.content.role if event.content else 'None'}: {text}...")

📝 Session contains:
  user: I'm interested in green technology, especially fusion, fissi...
  model: (Non-Text Part)......
  user: (Non-Text Part)......
  model: Recent breakthroughs in sustainable energy include advanceme...


#### d. Add Session to Memory

Perfect! The session contains our conversation. Now we're ready to transfer it to memory. Call add_session_to_memory() and pass the session object. This ingests the conversation into the memory store, making it available for future searches.

In [13]:
# This is the key method!
await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")

✅ Session added to memory!


Test the memory.

In [14]:
await run_session(runner, "what area of sustainability am I interested?", "interest test")


### Session: interest test

User > what area of sustainability am I interested?


Model: > You are interested in green technology, specifically fusion, fission, and battery technologies.


### 3.3: Now, let's add more to the conversation! 
Has to the same session-id: conversation-01

In [15]:
# User tells agent about other details on the interest
await run_session(
    runner,
    "I'm also interested in what companies are ahead in these green technologies. List these company names, how their technology stands out, and why the company is successful.",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > I'm also interested in what companies are ahead in these green technologies. List these company names, how their technology stands out, and why the company is successful.


Model: > Here are some of the companies leading in green technology sectors:

**Fusion Energy:**
*   **TAE Technologies:** Focuses on an aneutronic fusion approach using advanced beam-driven field-reversed configuration (FRC) technology. Their success is driven by significant private investment and a unique approach to achieving fusion.
*   **Helion Energy:** Developing a pulsed non-ignition fusion approach aiming for a device that generates more electricity than it consumes. They have secured substantial funding and are backed by prominent investors.
*   **Commonwealth Fusion Systems (CFS):** A spin-off from MIT, CFS is developing compact fusion devices using high-temperature superconducting (HTS) magnets. Their innovation in magnet technology allows for smaller, potentially more cost-effective fusion reactors. They have strong backing from major investors.
*   **General Fusion:** Pursuing a magnetized target fusion (MTF) approach, which uses lasers or other drivers to compress a plas

Manually save session to memory

In [16]:
# Manually save the session to memory
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

await memory_service.add_session_to_memory(session)

print("✅ Green Technology session saved to memory!")

✅ Green Technology session saved to memory!


See what's in the memory

In [17]:
# Let's see what's in the memory
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's interests?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 4 relevant memories

  [user]: I'm interested in green technology, especially fusion, fission, and battery tech...
  [ResearchMemoryAgent]: Recent breakthroughs in sustainable energy include advancements in fusion energy...
  [user]: I'm also interested in what companies are ahead in these green technologies. Lis...
  [ResearchMemoryAgent]: Here are some of the companies leading in green technology sectors:

**Fusion En...


### Session 4: Deployment

In [4]:
import os
import random
import time
import vertexai
from kaggle_secrets import UserSecretsClient
from vertexai import agent_engines

print("✅ Imports completed successfully")

✅ Imports completed successfully


#### a. Set up Cloud credential

In [5]:
# Set up Cloud Credentials in Kaggle
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

print("✅ Cloud credentials configured")

✅ Cloud credentials configured


#### b. Project-id

In [6]:
## Set your PROJECT_ID
PROJECT_ID = "myfirstagent-478223"  # TODO: Replace with your project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    raise ValueError("⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID.")

print(f"✅ Project ID set to: {PROJECT_ID}")

✅ Project ID set to: myfirstagent-478223


#### c. Create agent directory and necessary files

In [7]:
## Create simple agent - all code for the agent will live in this directory
!mkdir -p green_tech_agent

print(f"✅ Green Technology Agent directory created")

✅ Green Technology Agent directory created


In [8]:
%%writefile green_tech_agent/requirements.txt

google-adk
opentelemetry-instrumentation-google-genai

Writing green_tech_agent/requirements.txt


In [9]:
%%writefile green_tech_agent/.env

# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations#global-endpoint
GOOGLE_CLOUD_LOCATION="global"

# Set to 1 to use Vertex AI, or 0 to use Google AI Studio
GOOGLE_GENAI_USE_VERTEXAI=1

Writing green_tech_agent/.env


#### d. Create agent code
The code below does not include agent memory. To implement agent memory for deployment, copy every cell from Session 3 (Agent Memory) above into the bottom of the cell below. Ensure the entire code is in a single cell so that it is written to the agent.py file completely.

In [10]:
%%writefile green_tech_agent/agent.py
from google.adk.agents import Agent
import vertexai
import os

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
)

# retry
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
    Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
    1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
    2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
    3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

Writing green_tech_agent/agent.py


#### e. Config file for resource

In [11]:
%%writefile green_tech_agent/.agent_engine_config.json
{
    "min_instances": 0,
    "max_instances": 1,
    "resource_limits": {"cpu": "1", "memory": "1Gi"}
}

Writing green_tech_agent/.agent_engine_config.json


#### f. Deployment regions

In [12]:
regions_list = ["europe-west1", "europe-west4", "us-east4", "us-west1"]
deployed_region = random.choice(regions_list)

print(f"✅ Selected deployment region: {deployed_region}")

✅ Selected deployment region: us-west1


#### g. Deply the agent!

In [13]:
!adk deploy agent_engine --project=$PROJECT_ID --region=$deployed_region green_tech_agent --agent_engine_config_file=green_tech_agent/.agent_engine_config.json

Staging all files in: /kaggle/working/green_tech_agent_tmp20251124_231729
Copying agent source code...
Copying agent source code complete.
Resolving files and dependencies...
Reading agent engine config from green_tech_agent/.agent_engine_config.json
Reading environment variables from /kaggle/working/green_tech_agent/.env
Ignoring GOOGLE_CLOUD_LOCATION in .env as `--region` was explicitly passed and takes precedence
Initializing Vertex AI...
Vertex AI initialized.
Created green_tech_agent_tmp20251124_231729/agent_engine_app.py
Files and dependencies resolved
Deploying to agent engine...
INFO:vertexai_genai.agentengines:Creating in-memory tarfile of source_packages
INFO:vertexai_genai.agentengines:Using agent framework: google-adk
INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=myfirstagent-478223.
INFO:vertexai_genai.agentengines:Agent Engine created. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engi

#### h. Retrieve the deployed agent

In [14]:
# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=deployed_region)

# Get the most recently deployed agent
agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

✅ Connected to deployed agent: projects/3222581548/locations/us-west1/reasoningEngines/7187903334957711360


In [16]:
async for item in remote_agent.async_stream_query(
    message="What is the latest progress in green technology, especially fusion, fission, and battery technology?",
    user_id="user_42",
):
    print(item)

{'model_version': 'gemini-2.5-flash-lite', 'content': {'parts': [{'function_call': {'id': 'adk-44469d47-1101-4e0e-a61f-ed12cbcd00a4', 'args': {'request': 'Latest progress in green technology, including fusion, fission, and battery technology.'}, 'name': 'ResearchAgent'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 18, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 18}], 'prompt_token_count': 142, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 142}], 'thoughts_token_count': 113, 'total_token_count': 273, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.822260856628418, 'invocation_id': 'e-e0eaa86a-1909-4f42-87cd-9f1c1329c054', 'author': 'ResearchCoordinator', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': '1aa1baa6-5490-4299-8ac7-0c77cf4dd4ab', 'timestamp': 1764026759.873439}
{'content': {'parts': [{'f

### IMPORTANT: Delete the deployed agent
To not continue to incur cost.

In [17]:
agent_engines.delete(resource_name=remote_agent.resource_name, force=True)

print("✅ Agent successfully deleted")

INFO:vertexai.agent_engines:Deleting AgentEngine resource: projects/3222581548/locations/us-west1/reasoningEngines/7187903334957711360
INFO:vertexai.agent_engines:Delete AgentEngine backing LRO: projects/3222581548/locations/us-west1/operations/5929188195915792384
INFO:vertexai.agent_engines:AgentEngine resource deleted: projects/3222581548/locations/us-west1/reasoningEngines/7187903334957711360


✅ Agent successfully deleted
